# Install dependencies

In [ ]:
!pip install pytorch_transformers flair seqeval

# Initialization

In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import sys

In [2]:
import logging


logger = logging.getLogger('sequence_tagger_bert')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger.handlers = []

fhandler = logging.handlers.TimedRotatingFileHandler(filename='logs.txt', when='midnight')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)
logger.addHandler(handler)

logger.setLevel(logging.DEBUG)

In [3]:
import torch

device = torch.device('cuda')
n_gpu = torch.cuda.device_count()

for i in range(n_gpu):
    print(torch.cuda.get_device_name(i))

Tesla V100-DGXS-16GB


In [4]:
CACHE_DIR = '../workdir/cache'
BATCH_SIZE = 16
#BATCH_SIZE = 8
PRED_BATCH_SIZE = 100
MAX_LEN = 128
MAX_N_EPOCHS = 4
#MAX_N_EPOCHS = 100
#MAX_N_EPOCHS = 50
#MAX_N_EPOCHS = 10
REDUCE_ON_PLATEAU = False
WEIGHT_DECAY = 0.01
LEARNING_RATE = 3e-5
#LEARNING_RATE = 1e-5
#LEARNING_RATE = 2e-5

# Load corpus

In [5]:
from flair.datasets import ColumnCorpus


data_folder = '../workdir/conll2003/eng'
corpus = ColumnCorpus(data_folder, 
                      {0 : 'text', 3 : 'ner'},
                      train_file='train.txt',
                      test_file='test.txt',
                      dev_file='dev.txt')

print(corpus.obtain_statistics())

2019-11-23 13:39:40,241 Reading data from ../workdir/conll2003/eng
2019-11-23 13:39:40,242 Train: ../workdir/conll2003/eng/train.txt
2019-11-23 13:39:40,243 Dev: ../workdir/conll2003/eng/dev.txt
2019-11-23 13:39:40,244 Test: ../workdir/conll2003/eng/test.txt
{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 14987,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 204567,
            "min": 1,
            "max": 113,
            "avg": 13.649629679055181
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 3684,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 46666,
            "min": 1,
            "max": 124,
            "avg": 12.667209554831704
        }
    },
    "DEV": {
        "dataset": "DEV",
        "total_number_of_documents": 346

# Create model

In [6]:
from bert_sequence_tagger import SequenceTaggerBert, BertForTokenClassificationCustom
from pytorch_transformers import BertTokenizer, BertForTokenClassification
import torch.nn as nn

from bert_sequence_tagger.bert_utils import make_bert_tag_dict_from_flair_corpus


bpe_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', 
                                              cache_dir=CACHE_DIR, 
                                              do_lower_case=False)

idx2tag, tag2idx = make_bert_tag_dict_from_flair_corpus(corpus)

model = BertForTokenClassificationCustom.from_pretrained('bert-base-cased', 
                                                         cache_dir=CACHE_DIR, 
                                                         num_labels=len(tag2idx)).cuda()

seq_tagger = SequenceTaggerBert(bert_model=model, bpe_tokenizer=bpe_tokenizer, 
                                idx2tag=idx2tag, tag2idx=tag2idx, max_len=MAX_LEN)

# Train model

In [ ]:
from torch.utils.data import RandomSampler, SequentialSampler

from bert_sequence_tagger.bert_utils import create_loader_from_flair_corpus, get_model_parameters, get_parameters_without_decay
from bert_sequence_tagger.model_trainer_bert import ModelTrainerBert

from pytorch_transformers import AdamW, WarmupLinearSchedule
#from torch.optim.lr_scheduler import ReduceLROnPlateau

from bert_sequence_tagger.metrics import f1_entity_level, f1_token_level


train_dataloader = create_loader_from_flair_corpus(corpus.train, 
                                                   RandomSampler, 
                                                   batch_size=BATCH_SIZE)
val_dataloader = create_loader_from_flair_corpus(corpus.dev,
                                                 SequentialSampler,
                                                 batch_size=PRED_BATCH_SIZE)

optimizer = AdamW(get_model_parameters(model), 
                  lr=LEARNING_RATE, betas=(0.9, 0.999), 
                  eps =1e-6, weight_decay=0.01, correct_bias=True)
lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0.1, 
                                    t_total=(len(corpus.train) / BATCH_SIZE)*MAX_N_EPOCHS)
#lr_scheduler = ReduceLROnPlateau(optimizer)

trainer = ModelTrainerBert(model=seq_tagger, 
                           optimizer=optimizer, 
                           lr_scheduler=lr_scheduler,
                           train_dataloader=train_dataloader, 
                           val_dataloader=val_dataloader,
                           update_scheduler='es',
                           keep_best_model=False,
                           restore_bm_on_lr_change=False,
                           max_grad_norm=1.,
                           validation_metrics=[f1_entity_level],
                           decision_metric=lambda metrics: -metrics[1])

trainer.train(epochs=MAX_N_EPOCHS)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

2019-11-23 13:42:40,110 - sequence_tagger_bert - INFO - Current learning rate: 2.2970520331496137e-05
2019-11-23 13:42:40,112 - sequence_tagger_bert - INFO - Train loss: 0.09419015559311086
2019-11-23 13:42:49,310 - sequence_tagger_bert - INFO - Validation loss: 0.03420208178100769
2019-11-23 13:42:49,312 - sequence_tagger_bert - INFO - Validation metrics: (0.94098829867834,)


Epoch:  25%|██▌       | 1/4 [02:38<07:54, 158.04s/it]

In [13]:
test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                  SequentialSampler,
                                                  batch_size=PRED_BATCH_SIZE)

_, __, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')

2019-11-23 13:20:09,423 - sequence_tagger_bert - INFO - Entity-level f1: 0.911490409994721
2019-11-23 13:20:09,424 - sequence_tagger_bert - INFO - Token-level f1: 0.9273060476481368


In [ ]:
(0.9143007822800387, 0.9306361914074436)